In [21]:
from typing import Optional
import pandas as pd
import yaml

class FeatureCatalog:
    with open("../../data/features.yml", 'r') as file:
        data = yaml.safe_load(file)

    @classmethod
    def _parse_dict(cls, data, key):
        if key in data:
            return data[key]
        else:
            return ""

    @classmethod
    def _parse_list(cls, data):
        return [item for item in data if item not in ["", " ", None]]

    @classmethod
    def _print_table(cls, table):
        pd.set_option("display.max_colwidth", None)
        display(pd.DataFrame(table))

    @classmethod
    def _format_parameters(cls, data):
        if "parameters" in data:
            formatted_parameters = []
            for parameter in data["parameters"]:
                name = cls._parse_dict(parameter, "name")
                description = cls._parse_dict(parameter, "description")
                formatted_parameters.append(f"{name}: {description}")

            return "\n".join(formatted_parameters)
        else:
            return ""

    @classmethod
    def show_tags(cls) -> None:
        table = {"Name": [], "Description": []}
        for tag in cls.data["tags"]:
            table["Name"].append(cls._parse_dict(tag, "name"))
            table["Description"].append(cls._parse_dict(tag, "description"))
            
        cls._print_table(table)

    @classmethod
    def show_features(cls, tag_name: Optional[str] = None) -> None:
        table = {"Name": [], "Tags": [], "Description": [], "Details": [], "When": [], "Parameters": []}
        for feature in cls.data["features"]:
            tags = cls._parse_list(feature["tags"])
            if tag_name is None or tag_name in tags:
                table["Name"].append(cls._parse_dict(feature, "name"))
                table["Tags"].append("\n".join(tags))
                table["Description"].append(cls._parse_dict(feature, "description"))
                table["Details"].append(cls._parse_dict(feature, "details"))
                table["When"].append(cls._parse_dict(feature, "when"))
                table["Parameters"].append(cls._format_parameters(feature))

        cls._print_table(table)

    @classmethod
    def show_feature(cls, feature_name: str) -> None:
        table = {"Name": [], "Tags": [], "Description": [], "Details": [], "When": [], "Parameters": []}
        for feature in cls.data["features"]:
            name = cls._parse_dict(feature, "name")
            if name.startswith(feature_name):
                table["Name"].append(name)
                table["Tags"].append("\n".join(cls._parse_list(feature["tags"])))
                table["Description"].append(cls._parse_dict(feature, "description"))
                table["Details"].append(cls._parse_dict(feature, "details"))
                table["When"].append(cls._parse_dict(feature, "when"))
                table["Parameters"].append(cls._format_parameters(feature))

        cls._print_table(table)

def catalog(tag: Optional[str] = None, feature: Optional[str] = None):
    if tag is None and feature is None:
        FeatureCatalog.show_tags()
    elif tag is None and feature is not None:
        FeatureCatalog.show_feature(feature_name=feature)
    else:
        FeatureCatalog.show_features(tag_name=tag)

In [22]:
catalog()

,Name,Description
0,rilevamento pattern,
1,ridondanza della serie,
2,regolarità delle time series,
3,conteggio,
4,statistica medie,
5,statistica di base,
6,trend,


In [23]:
catalog(tag="trend")

,Name,Tags,Description,Details,When,Parameters
0,"agg_linear_trend(x, param):",trend,Calcola una regressione lineare dei valori della serie temporale che sono stati aggregati su chunk tramite una funzione di aggrefazione fornita come input. I parametri controllano quali caratteristiche vengono restituite.,"È possibile specificare l'attributo da estrarre dalla regressione lineare, come ad esempio ""pvalue"", ""rvalue"", ""intercept"", ""slope"" o ""stderr"".","Utilizzabile per scoprire se la serie segue un trend lineare, la divisione in chunks permette ad esempio di fare kpi sulla stagionalità","attr: rappresenta l'attributo da estrarre dalla regressione lineare (""pvalue"", ""rvalue"", ""intercept"", ""slope"", ""stderr"").\nchunk_len: specifica quanti valori della serie temporale sono presenti in ciascun chunk.\nf_agg: specifica la funzione di aggregazione da utilizzare per aggregare i valori all'interno di ciascun chunk (""max"", ""min"", ""mean"", ""median"")."
1,"linear_trend(x, param)",trend,Calcola una regressione lineare per i valori della serie temporale rispetto alla sequenza da 0 a lunghezza della serie temporale meno uno.,"È possibile specificare l'attributo da estrarre dalla regressione lineare, come ad esempio ""pvalue"", ""rvalue"", ""intercept"", ""slope"" o ""stderr"".",Utilizzabile per scoprire se la serie segue un trend lineare,"attr: rappresenta l'attributo da estrarre dalla regressione lineare (""pvalue"", ""rvalue"", ""intercept"", ""slope"", ""stderr"").\nchunk_len: specifica quanti valori della serie temporale sono presenti in ciascun chunk.\nf_agg: specifica la funzione di aggregazione da utilizzare per aggregare i valori all'interno di ciascun chunk (""max"", ""min"", ""mean"", ""median"")."
2,longest_strike_above_mean(x),trend,Indica la lunghezza della sequenza consecutiva più lunga nella serie temporale in cui i valori sono superiori alla media della serie stessa.,,Analisi della Persistenza di Trend: Identificare il periodo più lungo in cui i valori rimangono costantemente al di sotto della media può rivelare la presenza di un trend negativo o una fase di declino nella serie temporale.,
3,longest_strike_below_mean(x),trend,Indica la lunghezza della sequenza consecutiva più lunga nella serie temporale in cui i valori sono inferiori alla media della serie stessa.,,Analisi della Persistenza di Trend: Identificare il periodo più lungo in cui i valori rimangono costantemente al di sotto della media può rivelare la presenza di un trend negativo o una fase di declino nella serie temporale.,
4,"augmented_dickey_fuller(x, param)",trend,"Verifica se una serie temporale è stazionaria (le sue proprietà statistiche, come la media e la varianza, rimangono costanti nel tempo). Restituisce il valore della statistica di test in base al parametro inviato",,Utilizzabile per scoprire la stazionarietà della serie temporale nel contesto di rilevamento di eventuali trend,"attr: può assumere valori come ""teststat"" (statistica del test), ""pvalue"" (valore p del test) o ""usedlag"" (numero di ritardi utilizzati nel test).\nautolag: specifica il criterio di selezione automatica del numero di ritardi e può assumere valori come ""AIC"" (criterio di informazione di Akaike), ""BIC"" (criterio di informazione bayesiano) o None."


In [24]:
catalog(feature="linear_trend")

,Name,Tags,Description,Details,When,Parameters
0,"linear_trend(x, param)",trend,Calcola una regressione lineare per i valori della serie temporale rispetto alla sequenza da 0 a lunghezza della serie temporale meno uno.,"È possibile specificare l'attributo da estrarre dalla regressione lineare, come ad esempio ""pvalue"", ""rvalue"", ""intercept"", ""slope"" o ""stderr"".",Utilizzabile per scoprire se la serie segue un trend lineare,"attr: rappresenta l'attributo da estrarre dalla regressione lineare (""pvalue"", ""rvalue"", ""intercept"", ""slope"", ""stderr"").\nchunk_len: specifica quanti valori della serie temporale sono presenti in ciascun chunk.\nf_agg: specifica la funzione di aggregazione da utilizzare per aggregare i valori all'interno di ciascun chunk (""max"", ""min"", ""mean"", ""median"")."
